## Train step 1: Bootstrap from pretrained model

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'amharic', 'build') 

In [3]:
from load_pretrained_amharic_model import load_pretrained_amharic_model
model = load_pretrained_amharic_model(C, 0)

[NeMo W 2020-10-15 22:06:03 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 22:06:03 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 22:06:03 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 22:06:03 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 22:06:03 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

[NeMo I 2020-10-15 22:06:04 features:241] PADDING: 16
[NeMo I 2020-10-15 22:06:04 features:258] STFT using torch
loaded save/nemo_amharic/amharic_20201015_213218_437958_epoch=10.ckpt


In [4]:
import pytorch_lightning as pl
import os, datetime

model_save_dir='save/nemo_amharic'

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

trainer = pl.Trainer(gpus=[0], max_epochs=200, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [5]:
from ruamel.yaml import YAML
from omegaconf import DictConfig
config_path = 'amharic_16000.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest
model.set_trainer(trainer)
model.setup_training_data(train_data_config=params['model']['train_ds'])
model.setup_validation_data(val_data_config=params['model']['validation_ds'])
model.setup_optimization(optim_config=DictConfig(params['model']['optim']))

[NeMo I 2020-10-15 22:06:08 collections:173] Dataset loaded with 8368 files totalling 8.40 hours
[NeMo I 2020-10-15 22:06:08 collections:174] 29 files were filtered totalling 0.17 hours
[NeMo I 2020-10-15 22:06:08 collections:173] Dataset loaded with 933 files totalling 0.97 hours
[NeMo I 2020-10-15 22:06:08 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-15 22:06:08 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )
[NeMo I 2020-10-15 22:06:08 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f0b96ec7f90>" 
    will be used during training (effective maximum steps = 69733) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 69733
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.5]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 0.0001
     lr: 0.0001
     weight_decay: 0.001
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7f0b96ec7f90>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'val_loss',
  'reduce_on_plateau': False})

In [6]:
from reshuffle_samples import reshuffle_samples
reshuffle_samples(C)

In [7]:
trainer.fit(model)

[NeMo I 2020-10-15 22:06:20 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )
[NeMo I 2020-10-15 22:06:20 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f0b97069910>" 
    will be used during training (effective maximum steps = 69733) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 69733
    )


[NeMo W 2020-10-15 22:06:21 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
      warnings.warn(*args, **kwargs)
    

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18 M  
2 | decoder           | ConvASRDecoder                    | 241 K 
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     


[NeMo I 2020-10-15 22:06:22 wer:148] 
    
[NeMo I 2020-10-15 22:06:22 wer:149] reference:ወደዚህ ወደ መንዝ ምናምን
[NeMo I 2020-10-15 22:06:22 wer:150] decoded  :ወደዚህ ወደ መንዝ ምናምን


[NeMo W 2020-10-15 22:06:23 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
      "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
    


[NeMo I 2020-10-15 22:06:45 wer:148] 
    
[NeMo I 2020-10-15 22:06:45 wer:149] reference:እባክህ አለሁ አለሁ
[NeMo I 2020-10-15 22:06:45 wer:150] decoded  :እባክህ አለሁ አለሁ
[NeMo I 2020-10-15 22:07:08 wer:148] 
    
[NeMo I 2020-10-15 22:07:08 wer:149] reference:አዎ እንሂድ
[NeMo I 2020-10-15 22:07:08 wer:150] decoded  :አዎ እንሂድ
[NeMo I 2020-10-15 22:07:31 wer:148] 
    
[NeMo I 2020-10-15 22:07:31 wer:149] reference:በእዚያ ላይ ደግሞ ከአራት ቀን ከአምስት ቀን በኋላ ማለት እኮ ነው ምዝገባው የሚጀምረው
[NeMo I 2020-10-15 22:07:31 wer:150] decoded  :በእዚያ ላይ ደግሞ ከአራት ቀን ከምስት ቀን በኋላ ማለት እኮ ነው ምዝገባው የሚጀምረው
[NeMo I 2020-10-15 22:07:54 wer:148] 
    
[NeMo I 2020-10-15 22:07:54 wer:149] reference:ይሰማኛል አዎ ሥራ ላይ ነህ መሰለኝ አዎ እንደት ነህ ታዲያ እና ሥራ አሪፍ ነው እ
[NeMo I 2020-10-15 22:07:54 wer:150] decoded  :ይሰማኛል አዎ ሥራ ላይ ነህ መሰለኝ አዎ እንደት ነህ ታዲያ እና ሥራ አሪፍ ነው እ
[NeMo I 2020-10-15 22:08:17 wer:148] 
    
[NeMo I 2020-10-15 22:08:17 wer:149] reference:አዎ ሠራተኛሽ አለች አይደል ደህና ነች
[NeMo I 2020-10-15 22:08:17 wer:150] decoded  :አዎ ሠራተኛሽ አለች አይደል ደህና ነች
[NeMo 

...


Epoch 00198: saving model to /home/catskills/Desktop/openasr20/save/nemo_amharic/amharic_20201015_220606_440181_epoch=198.ckpt


[NeMo I 2020-10-16 07:11:05 wer:148] 
    
[NeMo I 2020-10-16 07:11:05 wer:149] reference:እ እ
[NeMo I 2020-10-16 07:11:05 wer:150] decoded  :እ እ
[NeMo I 2020-10-16 07:11:27 wer:148] 
    
[NeMo I 2020-10-16 07:11:27 wer:149] reference:አንቺ አመጣሺው
[NeMo I 2020-10-16 07:11:27 wer:150] decoded  :አንቺ አመጣሺው
[NeMo I 2020-10-16 07:11:49 wer:148] 
    
[NeMo I 2020-10-16 07:11:49 wer:149] reference:ጋሪ ተራው በ ወዴት ነው ብለሽ ነው የምጠይቂው በሮፓክ
[NeMo I 2020-10-16 07:11:49 wer:150] decoded  :ጋሪ ተራው ነኝ ወዴት ነው ብለሽ ነው የምጠይቂው በሮፓክ
[NeMo I 2020-10-16 07:12:12 wer:148] 
    
[NeMo I 2020-10-16 07:12:12 wer:149] reference:እሺ እሱም እኮ ነግሬሀለሁ አንተም በሚመችህ ሰዐት ከመጣህ እኔን ዝግጁ ነኝ ብየሀለሁ
[NeMo I 2020-10-16 07:12:12 wer:150] decoded  :እሺ እሱም በቃ ነግሬሀለሁ አንተም በሚመችህ ሰዐት ከመጣህ እኔን ዝግጁ ነኝ ብየሀለሁ
[NeMo I 2020-10-16 07:12:35 wer:148] 
    
[NeMo I 2020-10-16 07:12:35 wer:149] reference:ይሰማሀል
[NeMo I 2020-10-16 07:12:35 wer:150] decoded  :ይሰማሀል
[NeMo I 2020-10-16 07:12:58 wer:148] 
    
[NeMo I 2020-10-16 07:12:58 wer:149] reference:ኦኬ
[Ne

[NeMo I 2020-10-16 07:13:21 wer:148] 
    
[NeMo I 2020-10-16 07:13:21 wer:149] reference:ወደዚህ ወደ መንዝ ምናምን
[NeMo I 2020-10-16 07:13:21 wer:150] decoded  :ወደዚህ ወደ መንዝ ምናምን
[NeMo I 2020-10-16 07:13:22 wer:148] 
    
[NeMo I 2020-10-16 07:13:22 wer:149] reference:አዎ ሁሌ ስምሽን ነው የምጠራው ያቺ ልጅ እንደው እንደት ሁና ይሆን እ ሰው ሀገር እንደው ሄዳ እያለች ነበር
[NeMo I 2020-10-16 07:13:22 wer:150] decoded  :አዎ ሁሌ ስምሽን ነው የምጠራው ያቺ ልጅ እንደው እንደት ሁና ይሆን እ ሰው ሀገር እንደው ሄዳ እያለች ነበር
[NeMo I 2020-10-16 07:13:22 wer:148] 
    
[NeMo I 2020-10-16 07:13:22 wer:149] reference:አዎ እየወጣ ነዋ
[NeMo I 2020-10-16 07:13:22 wer:150] decoded  :አዎ እየወጣ ነዋ
[NeMo I 2020-10-16 07:13:22 wer:148] 
    
[NeMo I 2020-10-16 07:13:22 wer:149] reference:እ
[NeMo I 2020-10-16 07:13:22 wer:150] decoded  :እ
[NeMo I 2020-10-16 07:13:22 wer:148] 
    
[NeMo I 2020-10-16 07:13:22 wer:149] reference:አይ ችግ እንደምን ችግር አለ ታዲያ በደንብ ደግሞ ሪቫይዝ ታደርጊና በሚቀጥለው በደንብ ትሰሪያለሻ
[NeMo I 2020-10-16 07:13:22 wer:150] decoded  :አይ ችግ እንደምን ችግር አለ ታዲያ በደንብ ደግሞ ሪቫይዝ ታደርጊና በሚቀጥለው በደንብ 


Epoch 00199: saving model to /home/catskills/Desktop/openasr20/save/nemo_amharic/amharic_20201015_220606_440181_epoch=199.ckpt
Saving latest checkpoint..


1

In [9]:
from glob import glob
version='113'
print("version", version)
C1 = Cfg('NIST', 16000, 'amharic', 'dev', version)
files=list(sorted(glob(f'{C1.audio_split_dir}/*.wav')))
translations=model.transcribe(paths2audio_files=files, batch_size=16)


version 113
[NeMo I 2020-10-16 07:53:46 collections:173] Dataset loaded with 9341 files totalling 259472.22 hours
[NeMo I 2020-10-16 07:53:46 collections:174] 0 files were filtered totalling 0.00 hours


NameError: name 'package_DEV' is not defined

In [10]:
from package_DEV import package_DEV

In [11]:
package_DEV(C1, files, translations)

[NeMo W 2020-10-16 08:51:23 nemo_logging:349] /home/catskills/Desktop/openasr20/package_DEV.py:25: RuntimeWarning: invalid value encountered in true_divide
      DT = tdur*token_sizes/token_sizes.sum()
    


wrote ../../catskills_openASR20_dev_amharic_113.tgz
